In [ ]:
#!/usr/bin/env python3
"""
PyBackendPro 挑战练习：构建专业级日志系统

🎯 目标：设计一个微服务架构的日志管理系统
💡 难度：中级 -> 高级
🏆 通过标准：实现完整的日志链路追踪

任务清单：
✅ 1. 多模块层次化日志配置
✅ 2. 不同环境的配置切换
✅ 3. 性能监控和错误追踪  
✅ 4. 结构化日志输出
✅ 5. 日志轮转和归档
"""

import logging
import logging.handlers
import sys
import json
import time
import traceback
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, Any, Optional
from contextlib import contextmanager
from functools import wraps
import os

# =====================================
# 挑战 1: 设计专业级日志配置系统
# =====================================

class LoggingConfig:
    """
    挑战：设计一个支持多环境的日志配置系统
    
    要求：
    1. 支持 development/testing/production 三种环境
    2. 每种环境有不同的日志级别和输出格式
    3. 生产环境支持日志轮转
    4. 开发环境支持彩色输出
    """
    
    ENVIRONMENTS = {
        'development': {
            'level': logging.DEBUG,
            'console_level': logging.DEBUG,
            'file_level': logging.INFO,
            'format': '[%(asctime)s] [%(name)20s] [%(levelname)8s] %(message)s',
            'colored': True,
            'include_caller': True
        },
        'testing': {
            'level': logging.INFO,
            'console_level': logging.WARNING,
            'file_level': logging.INFO,
            'format': '[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s',
            'colored': False,
            'include_caller': False
        },
        'production': {
            'level': logging.WARNING,
            'console_level': logging.ERROR,
            'file_level': logging.WARNING,
            'format': '%(asctime)s [%(process)d] [%(name)s:%(lineno)d] [%(levelname)s] %(message)s',
            'colored': False,
            'include_caller': True,
            'rotation': True
        }
    }
    
    @classmethod
    def setup(cls, environment: str = 'development', app_name: str = 'microservice'):
        """
        配置日志系统
        
        Args:
            environment: 环境名称 (development/testing/production)
            app_name: 应用名称，用于日志文件命名
        """
        if environment not in cls.ENVIRONMENTS:
            raise ValueError(f"未知环境: {environment}，支持的环境: {list(cls.ENVIRONMENTS.keys())}")
        
        config = cls.ENVIRONMENTS[environment]
        
        # 清除现有配置
        logging.root.handlers.clear()
        logging.root.setLevel(config['level'])
        
        # 创建日志目录
        log_dir = Path('logs')
        log_dir.mkdir(exist_ok=True)
        
        # 1. 控制台处理器
        console_handler = logging.StreamHandler(sys.stdout)
        console_handler.setLevel(config['console_level'])
        
        if config.get('colored') and environment == 'development':
            console_handler.setFormatter(cls._get_colored_formatter())
        else:
            console_handler.setFormatter(logging.Formatter(config['format']))
        
        logging.root.addHandler(console_handler)
        
        # 2. 文件处理器
        if config.get('rotation') and environment == 'production':
            # 生产环境：使用轮转文件处理器
            file_handler = logging.handlers.RotatingFileHandler(
                log_dir / f'{app_name}.log',
                maxBytes=10*1024*1024,  # 10MB
                backupCount=5,
                encoding='utf-8'
            )
        else:
            # 开发/测试环境：普通文件处理器
            file_handler = logging.FileHandler(
                log_dir / f'{app_name}_{environment}.log',
                encoding='utf-8'
            )
        
        file_handler.setLevel(config['file_level'])
        file_formatter = logging.Formatter(
            f"{config['format']} | PID:%(process)d | Thread:%(thread)d"
        )
        file_handler.setFormatter(file_formatter)
        logging.root.addHandler(file_handler)
        
        # 3. 错误文件处理器（只记录ERROR以上）
        error_handler = logging.FileHandler(
            log_dir / f'{app_name}_errors.log',
            encoding='utf-8'
        )
        error_handler.setLevel(logging.ERROR)
        error_handler.setFormatter(file_formatter)
        logging.root.addHandler(error_handler)
        
        print(f"✅ 日志系统已配置 - 环境: {environment}")
        print(f"📁 日志目录: {log_dir.absolute()}")
    
    @staticmethod
    def _get_colored_formatter():
        """彩色格式器 - 仅开发环境使用"""
        class ColoredFormatter(logging.Formatter):
            COLORS = {
                'DEBUG': '\033[96m',    # 青色
                'INFO': '\033[92m',     # 绿色
                'WARNING': '\033[93m',  # 黄色
                'ERROR': '\033[91m',    # 红色
                'CRITICAL': '\033[95m\033[1m',  # 紫色加粗
                'RESET': '\033[0m'
            }
            
            def format(self, record):
                color = self.COLORS.get(record.levelname, '')
                reset = self.COLORS['RESET']
                record.levelname = f"{color}{record.levelname:8}{reset}"
                return super().format(record)
        
        return ColoredFormatter('[%(asctime)s] [%(name)20s] [%(levelname)s] %(message)s')

# =====================================
# 挑战 2: 实现请求链路追踪
# =====================================

class RequestTracker:
    """
    挑战：实现分布式系统的请求链路追踪
    
    要求：
    1. 每个请求生成唯一的 trace_id
    2. 所有相关日志包含 trace_id
    3. 支持嵌套的操作追踪
    4. 记录请求处理时间
    """
    
    _context = {}  # 模拟线程本地存储
    
    @classmethod
    @contextmanager
    def trace_request(cls, request_id: str, operation: str):
        """
        请求追踪上下文管理器
        
        Args:
            request_id: 请求唯一标识
            operation: 操作名称
        """
        import uuid
        trace_id = str(uuid.uuid4())[:8]
        
        # 保存上下文
        old_context = cls._context.copy()
        cls._context.update({
            'trace_id': trace_id,
            'request_id': request_id,
            'operation': operation,
            'start_time': time.time()
        })
        
        logger = logging.getLogger(f"trace.{operation}")
        logger.info(f"🚀 开始处理请求", extra=cls._get_extra())
        
        try:
            yield trace_id
        except Exception as e:
            logger.error(f"💥 请求处理失败: {e}", extra=cls._get_extra())
            raise
        finally:
            duration = time.time() - cls._context['start_time']
            logger.info(f"✅ 请求处理完成 耗时:{duration:.3f}s", extra=cls._get_extra())
            
            # 恢复上下文
            cls._context = old_context
    
    @classmethod
    def _get_extra(cls) -> Dict[str, Any]:
        """获取追踪信息"""
        return {
            'trace_id': cls._context.get('trace_id'),
            'request_id': cls._context.get('request_id'),
            'operation': cls._context.get('operation')
        }
    
    @classmethod
    def get_logger(cls, name: str) -> logging.Logger:
        """获取带追踪信息的logger"""
        logger = logging.getLogger(name)
        
        # 添加自定义格式器来包含追踪信息
        class TraceFormatter(logging.Formatter):
            def format(self, record):
                if hasattr(record, 'trace_id') and record.trace_id:
                    record.msg = f"[{record.trace_id}] {record.msg}"
                return super().format(record)
        
        # 为所有handler添加追踪格式器
        for handler in logger.handlers:
            if not isinstance(handler.formatter, TraceFormatter):
                original_format = handler.formatter._fmt if handler.formatter else '%(message)s'
                handler.setFormatter(TraceFormatter(original_format))
        
        return logger

# =====================================
# 挑战 3: 性能监控装饰器
# =====================================

def performance_monitor(threshold_ms: float = 1000.0):
    """
    挑战：创建性能监控装饰器
    
    要求：
    1. 记录函数执行时间
    2. 超过阈值时记录WARNING
    3. 记录函数参数和返回值（可选）
    4. 支持异步函数
    """
    def decorator(func):
        logger = logging.getLogger(f"perf.{func.__module__}.{func.__name__}")
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            start_time = time.time()
            func_signature = f"{func.__name__}({len(args)} args, {len(kwargs)} kwargs)"
            
            logger.debug(f"🔧 开始执行: {func_signature}")
            
            try:
                result = func(*args, **kwargs)
                
                duration_ms = (time.time() - start_time) * 1000
                
                if duration_ms > threshold_ms:
                    logger.warning(f"🐌 慢查询: {func_signature} 耗时 {duration_ms:.2f}ms")
                else:
                    logger.info(f"⚡ 完成执行: {func_signature} 耗时 {duration_ms:.2f}ms")
                
                return result
                
            except Exception as e:
                duration_ms = (time.time() - start_time) * 1000
                logger.error(f"💥 执行失败: {func_signature} 耗时 {duration_ms:.2f}ms 错误: {e}")
                raise
        
        return wrapper
    return decorator

# =====================================
# 挑战 4: 结构化日志处理器
# =====================================

class StructuredLogger:
    """
    挑战：实现结构化日志系统
    
    要求：
    1. 支持JSON格式输出
    2. 自动添加环境信息
    3. 支持自定义字段
    4. 便于ELK stack处理
    """
    
    def __init__(self, name: str, service_name: str = "unknown"):
        self.logger = logging.getLogger(name)
        self.service_name = service_name
        self.environment = os.getenv('ENV', 'development')
    
    def _create_log_record(self, level: str, message: str, **kwargs) -> Dict[str, Any]:
        """创建结构化日志记录"""
        record = {
            '@timestamp': datetime.utcnow().isoformat() + 'Z',
            'level': level,
            'message': message,
            'service': self.service_name,
            'environment': self.environment,
            'logger': self.logger.name
        }
        
        # 添加追踪信息
        if RequestTracker._context:
            record.update({
                'trace_id': RequestTracker._context.get('trace_id'),
                'request_id': RequestTracker._context.get('request_id'),
                'operation': RequestTracker._context.get('operation')
            })
        
        # 添加自定义字段
        record.update(kwargs)
        
        return record
    
    def info(self, message: str, **kwargs):
        """记录INFO级别的结构化日志"""
        record = self._create_log_record('INFO', message, **kwargs)
        self.logger.info(json.dumps(record, ensure_ascii=False))
    
    def error(self, message: str, error: Exception = None, **kwargs):
        """记录ERROR级别的结构化日志"""
        record = self._create_log_record('ERROR', message, **kwargs)
        
        if error:
            record.update({
                'error_type': type(error).__name__,
                'error_message': str(error),
                'traceback': traceback.format_exc()
            })
        
        self.logger.error(json.dumps(record, ensure_ascii=False))
    
    def warning(self, message: str, **kwargs):
        """记录WARNING级别的结构化日志"""
        record = self._create_log_record('WARNING', message, **kwargs)
        self.logger.warning(json.dumps(record, ensure_ascii=False))

# =====================================
# 挑战实战：构建微服务示例
# =====================================

class UserService:
    """用户服务 - 演示完整的日志实践"""
    
    def __init__(self):
        self.logger = RequestTracker.get_logger(f"{__name__}.{self.__class__.__name__}")
        self.struct_logger = StructuredLogger(f"{__name__}.structured", "user-service")
    
    @performance_monitor(threshold_ms=500)
    def create_user(self, username: str, email: str) -> Dict[str, Any]:
        """创建用户"""
        self.logger.info(f"创建用户请求: {username}")
        
        # 模拟验证
        if not username or not email:
            self.logger.error("用户创建失败: 缺少必要参数")
            raise ValueError("用户名和邮箱不能为空")
        
        # 模拟数据库操作（耗时）
        time.sleep(0.1)
        
        user_data = {
            'id': f"user_{int(time.time())}",
            'username': username,
            'email': email,
            'created_at': datetime.now().isoformat()
        }
        
        # 结构化日志记录
        self.struct_logger.info(
            "用户创建成功",
            user_id=user_data['id'],
            username=username,
            action="create_user"
        )
        
        return user_data
    
    @performance_monitor(threshold_ms=200)
    def get_user(self, user_id: str) -> Optional[Dict[str, Any]]:
        """获取用户信息"""
        self.logger.debug(f"查询用户: {user_id}")
        
        # 模拟数据库查询
        time.sleep(0.05)
        
        if user_id.startswith("user_"):
            user_data = {
                'id': user_id,
                'username': f"user_{user_id.split('_')[1]}",
                'email': f"user{user_id.split('_')[1]}@example.com"
            }
            
            self.logger.info(f"用户查询成功: {user_id}")
            return user_data
        else:
            self.logger.warning(f"用户不存在: {user_id}")
            return None

class PaymentService:
    """支付服务 - 演示错误处理和监控"""
    
    def __init__(self):
        self.logger = RequestTracker.get_logger(f"{__name__}.{self.__class__.__name__}")
        self.struct_logger = StructuredLogger(f"{__name__}.structured", "payment-service")
    
    @performance_monitor(threshold_ms=2000)
    def process_payment(self, user_id: str, amount: float) -> Dict[str, Any]:
        """处理支付"""
        self.logger.info(f"处理支付: 用户={user_id}, 金额={amount}")
        
        # 模拟支付网关调用（可能失败）
        time.sleep(0.3)
        
        if amount > 10000:
            error = ValueError(f"支付金额过大: {amount}")
            self.struct_logger.error(
                "支付处理失败",
                error=error,
                user_id=user_id,
                amount=amount,
                reason="amount_exceeded"
            )
            raise error
        
        payment_id = f"pay_{int(time.time())}"
        
        self.struct_logger.info(
            "支付处理成功",
            payment_id=payment_id,
            user_id=user_id,
            amount=amount,
            action="process_payment"
        )
        
        return {
            'payment_id': payment_id,
            'status': 'success',
            'amount': amount
        }

def main():
    """主函数 - 运行挑战演示"""
    print("🚀 PyBackendPro 日志挑战实战")
    print("="*60)
    
    # 1. 配置不同环境的日志系统
    environments = ['development', 'testing', 'production']
    
    for env in environments:
        print(f"\n🔧 配置 {env} 环境:")
        LoggingConfig.setup(env, 'microservice-demo')
        
        # 测试日志输出
        test_logger = logging.getLogger(f'test.{env}')
        test_logger.debug(f"{env} 环境调试信息")
        test_logger.info(f"{env} 环境正常运行")
        test_logger.warning(f"{env} 环境警告信息")
        test_logger.error(f"{env} 环境错误信息")
    
    # 2. 重新配置为开发环境进行演示
    print(f"\n{'='*60}")
    print("🎯 切换到开发环境进行完整演示")
    LoggingConfig.setup('development', 'challenge-demo')
    
    # 3. 演示微服务请求处理
    user_service = UserService()
    payment_service = PaymentService()
    
    # 模拟用户注册和支付流程
    request_scenarios = [
        ("用户注册流程", lambda: user_service.create_user("alice", "alice@example.com")),
        ("用户查询流程", lambda: user_service.get_user("user_123456")),
        ("正常支付流程", lambda: payment_service.process_payment("user_123456", 99.99)),
        ("异常支付流程", lambda: payment_service.process_payment("user_123456", 15000.00))
    ]
    
    for scenario_name, scenario_func in request_scenarios:
        with RequestTracker.trace_request(f"req_{int(time.time())}", scenario_name):
            try:
                result = scenario_func()
                print(f"✅ {scenario_name} 成功")
            except Exception as e:
                print(f"❌ {scenario_name} 失败: {e}")
    
    print(f"\n{'='*60}")
    print("🎉 挑战完成！检查 logs/ 目录查看日志文件")
    print("💡 在生产环境中，这些日志可以被 ELK/Prometheus 等工具收集分析")
    print("🏆 你已经掌握了专业级的Python日志管理技能！")

if __name__ == "__main__":
    main() 